# Lesson 2: Pydantic Basics

In this lesson, you'll learn the fundamentals of Pydantic models for data validation using a customer support system as your example application. You'll see how to define data models, validate user input, and handle validation errors gracefully.

By the end of this lesson, you'll be able to:
- Create Pydantic models to validate user input data
- Handle validation errors with proper error handling
- Use optional fields and field constraints in your models
- Work with JSON data validation methods

---

### Resumo das Ideias Chave do Notebook

O notebook ensina, de forma prática, como usar Pydantic para garantir a qualidade dos dados que entram em um sistema. A ideia central é simples: **"Declare a estrutura dos seus dados com classes Python e deixe o Pydantic fazer o trabalho pesado de validação."**

Os principais conceitos que ele aborda são:

1.  **Declaração Estruturada de Dados com `BaseModel`**: Em vez de trabalhar com dicionários Python "soltos" e ter que verificar manualmente se as chaves existem (`'name' in data`), você define uma classe que herda de `pydantic.BaseModel`. Essa classe se torna a "verdade única" sobre a estrutura de dados esperada. É auto-documentável e clara.

2.  **Validação e Coerção Automática de Tipos**: O Pydantic não apenas verifica se um campo é do tipo correto, mas também tenta "forçar" (coerção) os dados para o tipo esperado, se for razoável. O notebook demonstra isso perfeitamente:
    * Uma string de data como `"2025-12-31"` é automaticamente convertida para um objeto `datetime.date`.
    * Um `order_id` passado como a string `"12345"` é convertido para o inteiro `12345`.
    * Ele também usa tipos especializados como `EmailStr` para validações complexas prontas para uso.

3.  **Tratamento de Erros Centralizado com `ValidationError`**: Quando os dados não podem ser validados ou convertidos, o Pydantic não quebra seu programa de forma inesperada. Ele levanta uma única exceção, `ValidationError`, que contém uma lista detalhada de todos os erros encontrados. A função `validate_user_input` do notebook é um exemplo perfeito de como capturar essa exceção e apresentar os erros de forma amigável, tornando a aplicação muito mais robusta.

4.  **Flexibilidade com Campos Opcionais e Restrições**: Nem todos os dados são obrigatórios. O notebook mostra como usar `Optional` para campos que podem ou não estar presentes. Além disso, introduz o `Field` para adicionar regras de negócio diretamente na definição do modelo (ex: `order_id` deve ser um número entre 10000 e 99999). Isso move a lógica de validação para perto da definição do dado, tornando o código mais limpo.

5.  **Manipulação Eficiente de JSON**: A lição mostra a forma correta e moderna de lidar com dados JSON. Em vez de fazer `json.loads(data)` e depois passar o dicionário para o modelo, você pode usar o método `UserInput.model_validate_json(json_data)`. Isso é mais performático e direto, combinando o parsing do JSON e a validação em um único passo.

---

### Aplicação em LangChain e LangGraph

Agora, a parte mais importante: por que tudo isso é fundamental para LangChain?

LLMs são, por natureza, geradores de texto não estruturado. Para construir aplicações confiáveis, precisamos forçar essa saída de texto a se conformar com uma estrutura de dados bem definida. **Pydantic é a principal ferramenta para criar essa estrutura.**

1.  **Structured Output (Saída Estruturada)**:
    * **Cenário**: Você pede a um LLM para extrair informações de um e-mail de cliente e quer a resposta em um formato JSON específico.
    * **Aplicação Pydantic**: Você define um `BaseModel` Pydantic com os campos que deseja (ex: `nome_cliente`, `numero_pedido`, `sentimento`). Em LangChain, você usa o `PydanticOutputParser` e passa seu modelo Pydantic a ele. O LangChain automaticamente gera um prompt que instrui o LLM a formatar sua resposta de acordo com o esquema do seu modelo Pydantic. Depois, ele valida a saída do LLM e a entrega para você como uma instância da sua classe Pydantic, já validada e com os tipos corretos. Isso garante que a saída do LLM seja sempre utilizável pelo resto do seu código.

2.  **Definição de Ferramentas (Tools) para Agentes**:
    * **Cenário**: Você está construindo um agente que pode usar ferramentas, como uma função para buscar informações de um pedido em um banco de dados.
    * **Aplicação Pydantic**: A melhor prática é definir os argumentos que sua ferramenta espera usando um `BaseModel` Pydantic. Isso serve como um "esquema" para o LLM. O agente usará esse esquema para entender que, para chamar a ferramenta `buscar_pedido`, ele precisa fornecer um argumento chamado `order_id` que deve ser um inteiro. Isso torna a chamada de funções (function calling) muito mais confiável.

3.  **Definição do Estado em LangGraph**:
    * **Cenário**: Em LangGraph, você constrói grafos de execução onde o estado (informações) flui de um nó para outro.
    * **Aplicação Pydantic**: A forma mais robusta de definir o `State` do seu grafo é usando um `BaseModel` Pydantic (ou um `TypedDict`, que é similar). Cada campo no seu modelo Pydantic representa uma parte do estado da aplicação (ex: `email_original`, `resumo`, `dados_extraidos`). Isso funciona como um **contrato de dados** entre os nós do grafo. Se um nó é responsável por preencher o campo `resumo`, o Pydantic garante que ele será uma string, evitando que o próximo nó quebre ao tentar usá-lo.

Em resumo, o que você aprendeu neste notebook é a base para transformar protótipos de LangChain que funcionam "na maioria das vezes" em sistemas de produção que são **confiáveis, previsíveis e fáceis de manter**.

In [1]:
# Import libraries needed for the lesson
from typing import Any
from pydantic import BaseModel, ValidationError, EmailStr
import json

### Define a UserInput Pydantic model and populate it with data

In [2]:
# Create a Pydantic model for validating user input
class UserInput(BaseModel):
    """A Pydantic model for validating user input data.
    
    This class represents user input containing personal information and a query.
    It uses Pydantic for automatic validation and serialization of the input data.
    
    Attributes:
        name (str): The full name of the user. Must be a non-empty string.
        email (EmailStr): A valid email address. Automatically validated for
        proper email format using Pydantic's EmailStr type.
        query (str): The user's question or request. Must be a non-empty string.
    
    Example:
        >>> user_data = UserInput(
        ...     name="João Silva",
        ...     email="joao.silva@email.com",
        ...     query="Como posso aprender Python?"
        ... )
        >>> print(user_data.name)
        João Silva
        >>> print(user_data.email)
        joao.silva@email.com
    
    Raises:
        ValidationError: If any of the provided values don't meet the validation
            criteria (e.g., invalid email format, empty strings).
    
    Note:
        This class inherits from Pydantic's BaseModel, which provides automatic
        validation, serialization, and deserialization capabilities.
    """
    name: str
    email: EmailStr
    query: str

In [3]:
# Create a model instance
user_input = UserInput(
    name="Joe User", 
    email="joe.user@example.com", 
    query="I forgot my password."
)
print(user_input)

name='Joe User' email='joe.user@example.com' query='I forgot my password.'


### Note: the following cell will produce a validation error. You can correct the error by following along with the video, or just proceed with the rest of the notebook as cells below do not depend on this cell. 

In [4]:
# Attempt to create another model instance with an invalid email
user_input = UserInput(
    name="Joe User", 
    email="not-an-email", 
    query="I forgot my password."
)
print(user_input)

ValidationError: 1 validation error for UserInput
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not-an-email', input_type=str]

### Define a function for error handling and try different inputs

In [5]:
# Define a function to handle user input validation safely
def validate_user_input(input_data: dict[str, Any])-> UserInput | None:
    """Validate user input data and create a UserInput model instance.
    
    This function attempts to create and validate a UserInput instance from
    the provided dictionary data. It handles validation errors gracefully by
    printing user-friendly error messages and returning None on failure.
    
    Args:
        input_data: A dictionary containing user input data with keys 'name',
            'email', and 'query'. All values should be strings, with 'email'
            being a valid email address format.
    
    Returns:
        A validated UserInput instance if validation succeeds, None if
        validation fails due to invalid data format or missing required fields.
    
    Raises:
        ValidationError: Caught internally and converted to user-friendly
            error messages. The function will not propagate this exception.
    
    Example:
        >>> data = {
        ...     "name": "Maria Santos",
        ...     "email": "maria@example.com",
        ...     "query": "How to learn Python?"
        ... }
        >>> result = validate_user_input(data)
        ✅ Valid user input created:
        {
          "name": "Maria Santos",
          "email": "maria@example.com",
          "query": "How to learn Python?"
        }
        >>> isinstance(result, UserInput)
        True
        
        >>> invalid_data = {"name": "", "email": "invalid", "query": ""}
        >>> result = validate_user_input(invalid_data)
        ❌ Validation error occurred:
          - name: String should have at least 1 character
          - email: value is not a valid email address
        >>> result is None
        True
    
    Note:
        This function prints validation results directly to stdout. Consider
        using logging for production applications instead of print statements.
    """
    try:
        # Attempt to create a UserInput model instance from user input data
        user_input = UserInput(**input_data)
        print(f"✅ Valid user input created:")
        print(f"{user_input.model_dump_json(indent=2)}")
        return user_input
    except ValidationError as e:
        # Capture and display validation errors in a readable format
        print(f"❌ Validation error occurred:")
        for error in e.errors():
            print(f"  - {error['loc'][0]}: {error['msg']}")
        return None

O construtor da classe UserInput espera receber os argumentos assim:
```python
user_input = UserInput(name="Maria", email="maria@example.com", query="...")
```
Contudo os dados estão em um dicionário:
```python
input_data = {
    "name": "Maria",
    "email": "maria@example.com",
    "query": "..."
}
```
A linha UserInput(**input_data) é um atalho elegante que faz o Python **"desempacotar"** o dicionário input_data, transformando cada par de chave-valor em um argumento de palavra-chave.

Portanto, o código:
```python
user_input = UserInput(**input_data)
```
É exatamente equivalente a:
```python
user_input = UserInput(
    name=input_data["name"], 
    email=input_data["email"], 
    query=input_data["query"]
)
```


In [6]:
# Create an instance of UserInput using validate_user_input() function
input_data = {
    "name": "Joe User", 
    "email": "joe.user@example.com",
    "query": "I forgot my password."
}

user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I forgot my password."
}


In [7]:
# Attempt to create an instance of UserInput with missing query field
input_data = {
    "name": "Joe User", 
    "email": "joe.user@example.com"
}

user_input = validate_user_input(input_data)

❌ Validation error occurred:
  - query: Field required


### Update your UserInput data model with additional fields and experiment with different input data

In [8]:
# Import additional libraries for enhanced validation
from pydantic import Field
from typing import Optional
from datetime import date

# Define a new UserInput model with optional fields
class UserInput(BaseModel):
    """Enhanced Pydantic model for validating user input with optional fields.
    
    This class represents comprehensive user input data including personal 
    information, queries, and optional order details. It uses Pydantic's Field
    for advanced validation rules and constraints.
    
    Attributes:
        name (str): The full name of the user. Required field.
        email (EmailStr): A valid email address. Automatically validated for 
            proper email format.
        query (str): The user's question or request. Required field.
        order_id (Optional[int]): An optional 5-digit order number. Must be 
            between 10000-99999 (cannot start with 0). Defaults to None.
        purchase_date (Optional[date]): An optional purchase date. Accepts 
            ISO format dates (YYYY-MM-DD). Defaults to None.
    
    Examples:
        Basic usage with required fields only:
        >>> user = UserInput(
        ...     name="Ana Silva",
        ...     email="ana@example.com",
        ...     query="Preciso de ajuda com meu pedido"
        ... )
        
        Complete usage with all fields:
        >>> from datetime import date
        >>> user = UserInput(
        ...     name="Carlos Oliveira",
        ...     email="carlos@example.com",
        ...     query="Status do pedido",
        ...     order_id=12345,
        ...     purchase_date=date(2024, 1, 15)
        ... )
        
        Invalid order_id (will raise ValidationError):
        >>> user = UserInput(
        ...     name="Test User",
        ...     email="test@example.com", 
        ...     query="Test query",
        ...     order_id=123  # Too small, must be 5 digits
        ... )
    
    Raises:
        ValidationError: If any validation constraint is violated:
            - Invalid email format
            - order_id outside range 10000-99999
            - Invalid date format for purchase_date
    
    Note:
        This class leverages Pydantic's Field for advanced validation.
        Optional fields can be omitted and will default to None.
    """
    name: str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        None,
        description="5-digit order number (cannot start with 0)",
        ge=10000,
        le=99999
    )
    purchase_date: Optional[date] = None

### O Método `Field` do Pydantic
O Field é uma função especial do Pydantic que permite definir validações avançadas 
e metadados para os campos do modelo.

```python
campo: tipo = Field(valor_default, **configurações)
```
* Parâmetros Principais do `Field`
1. Default value
    ```python
    order_id: Optional[int] = Field(None, ...)  # None é o valor padrão
    ```
2. Description - documentação do campo
    ```python
    Field(description="5-digit order number (cannot start with 0)")
    ```
    * Propósito: Documenta o campo para desenvolvedores e documentação automática
    * Uso: Aparece em schemas JSON, documentação OpenAI, etc

3. ge - Greater than or Equal (Maior ou Igual)
    ```python
    Field(ge=10000)  # order_id >= 10000
    ```
    * Aplicação: Números(int, float)
    * Validação: Garante que o valor seja maior ou igual ao especificado

4. le - Less than or Equal (Menor ou Igual)
    ```python
    Field(le=99999)  # order_id <= 99999
    ```
    * Aplicação: Números(int, float)
    * Validação: Garante que o valor seja menor ou igual ao especificado

Outros Parâmetros Úteis do Field
Para Strings:
```python
name: str = Field(
    min_length=2,        # Mínimo 2 caracteres
    max_length=100,      # Máximo 100 caracteres
    regex=r"^[A-Za-z\s]+$"  # Apenas letras e espaços
)
```
Para Números:
```python
price: float = Field(
    gt=0,           # Greater than (maior que)
    lt=1000000,     # Less than (menor que)
    decimal_places=2  # Máximo 2 casas decimais
)
```
Para listas:
```python
tags: List[str] = Field(
    min_items=1,    # Mínimo 1 item
    max_items=10    # Máximo 10 itens
)
``` 
Conclusão: O `Field` transforma validação de dados de uma tarefa manual e propensa a erros em um processo declarativo e automático! 🚀

In [9]:
# Define a dictionary with required fields only
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I forgot my password."
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I forgot my password.",
  "order_id": null,
  "purchase_date": null
}


In [10]:
print(user_input)

name='Joe User' email='joe.user@example.com' query='I forgot my password.' order_id=None purchase_date=None


In [11]:
print(user_input.model_dump_json(indent=2))

{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I forgot my password.",
  "order_id": null,
  "purchase_date": null
}


In [12]:
user_input.model_dump()  # Dict Python

{'name': 'Joe User',
 'email': 'joe.user@example.com',
 'query': 'I forgot my password.',
 'order_id': None,
 'purchase_date': None}

In [13]:
# Define a dictionary with all fields including optional ones
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": date(2025, 12, 31)
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [14]:
# Define a dictionary with all fields and including additional ones
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": date(2025, 12, 31),
    "system_message": "logging status regarding order processing...",
    "iteration": 1 
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [15]:
print(user_input)

name='Joe User' email='joe.user@example.com' query='I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.' order_id=12345 purchase_date=datetime.date(2025, 12, 31)


In [16]:
# Create an instance of UserInput with valid data
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": "2025-12-31"
}

user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [17]:
# Define order_id as a string
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": "12345",
    "purchase_date": "2025-12-31"
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [18]:
# Define name field as an integer
input_data = {
    "name": 99999,
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": "2025-12-31"
}

# Validate the user input data
user_input = validate_user_input(input_data)

❌ Validation error occurred:
  - name: Input should be a valid string


### Try starting with JSON data as input

In [19]:
# Define user input as JSON data
json_data = '''
{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I bought a keyboard and mouse and was overcharged.",
    "order_id": 12345,
    "purchase_date": "2025-12-31"
}
'''

# Parse the JSON string into a Python dictionary
input_data = json.loads(json_data)
print("Parsed JSON:", input_data)

Parsed JSON: {'name': 'Joe User', 'email': 'joe.user@example.com', 'query': 'I bought a keyboard and mouse and was overcharged.', 'order_id': 12345, 'purchase_date': '2025-12-31'}


In [20]:
import json
# Parse the JSON string into a Python dictionary
input_data = json.loads(json_data)

# Converte o dicionário de volta para uma string JSON formatada
# O parâmetro 'indent=4' adiciona 4 espaços de indentação para cada nível
pretty_json = json.dumps(input_data, indent=4)

print(pretty_json)

{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I bought a keyboard and mouse and was overcharged.",
    "order_id": 12345,
    "purchase_date": "2025-12-31"
}


In [21]:
# Validate the user iput data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a keyboard and mouse and was overcharged.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [22]:
# Try different JSON input
json_data = '''
{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "My account has been locked for some reason.",
    "order_id": "12345",
    "purchase_date": "2025-12-31"
}
'''

# Parse the JSON into a Python dictionary
input_data = json.loads(json_data)
print("Parsed JSON:", input_data)

Parsed JSON: {'name': 'Joe User', 'email': 'joe.user@example.com', 'query': 'My account has been locked for some reason.', 'order_id': '12345', 'purchase_date': '2025-12-31'}


In [23]:
# Validate the customer support data from JSON with non-standard formats
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "My account has been locked for some reason.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


### Try the `model_validate_json` method

### Note: the following cell will produce a validation error. You can correct the error by following along with the video. 

In [24]:
#! Parse JSON and validate user input data in one step using model_validate_json method
user_input = UserInput.model_validate_json(json_data)
print(user_input.model_dump_json(indent=2))

{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "My account has been locked for some reason.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


---

## Conclusion

In this lesson, you learned how to use Pydantic models to validate user input for a customer support scenario. By defining clear data models and handling validation errors, you can ensure your code only works with well-formed data. This approach helps you build more robust and reliable applications, and sets the stage for more advanced validation and structured output in future lessons.